In [479]:
import pandas as pd
import pyreadstat 
import chardet
import numpy as np
import geopandas

In [480]:
#Import the csv
population = pd.read_csv('./Datasets/Populationbyregion.csv').drop_duplicates()
#Get rid of some of the forests that sit in different regions by adding them together
totals=population.groupby(['county','indicator'])['Value'].sum().reset_index()
#Pivot to get a wide rather than a narrow dataset
population_data = totals.pivot(index=['county'],columns='indicator', values='Value')
#The household size data is currently one significant figure, recalculate
population_data['Average Household size'] = population_data['Population, Total']/population_data['Number of Households']
population_data=population_data.reset_index()

In [481]:
mtf_battery_df, mtf_battery_meta = pyreadstat.read_dta('./Datasets/Kenya_MTF_data/MTF_HH_Battery_Data_Final.dta')
mtf_weight_df, mtf_weight_meta = pyreadstat.read_dta('./Datasets/Kenya_MTF_data/weight.dta')

In [482]:
with_weights = pd.merge(mtf_battery_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals=with_weights.groupby(['PARENT_KEY'])['pw_final', 'c_multiple'].max().reset_index()
sum_vals=with_weights.groupby(['PARENT_KEY'])['c_c_110', 'c_c_111'].sum().reset_index()
battery_hhs = pd.merge(max_vals,sum_vals,on = ['PARENT_KEY']).rename(columns={'c_c_110':'total_battery_capacity', 'c_c_111':'total_battery_voltage','c_multiple':'num_batteries'})

/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1801959063.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals=with_weights.groupby(['PARENT_KEY'])['pw_final', 'c_multiple'].max().reset_index()
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1801959063.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sum_vals=with_weights.groupby(['PARENT_KEY'])['c_c_110', 'c_c_111'].sum().reset_index()


In [483]:
mtf_core_df = pd.read_stata('./Datasets/Kenya_MTF_data/MTF_HH_Core_survey_Final_data_trimmed-2.dta', convert_categoricals=False).drop(columns=['enum_id', 'enum_foot','enum_ht','sup_id','tab_id','subdate','start_time'])
with_weights_survey = pd.merge(mtf_core_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['pw_final', 'a_a_1','b_b_6a','b_b_7'].max().reset_index().rename(columns={'a_a_1':'household_size','b_b_6a':'num_hhd_in_dwelling','b_b_7':'own_dwelling'})
mean_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['b_b_9','c_c_3','c_c_21','c_c_25ai','c_c_25aii'].mean().reset_index().rename(columns={'b_b_9':'rooms_occupied','c_c_3':'distance_to_grid','c_c_21':'avg_electricity_consumption_kwh','c_c_25ai':'worst_hours_electricity','c_c_25aii':'typical_hours_electricity'})
min_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])[ 'b_b_16','b_b_21','c_c_2','c_c_38','c_c_75','c_c_104'].min().reset_index().rename(columns={'b_b_16':'bank_account_held','b_b_21':'mobile_money_acc_held','c_c_2':'grid_connection','c_c_38':'mini_grid_connection','c_c_75':'generator_for_electricity','c_c_104':'rechargeable_battery_electricity'})
total_survey = pd.merge(pd.merge(max_vals_survey, mean_vals_survey, on = ['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc']), min_vals_survey, on = ['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])

/Users/rebecca.duke/miniforge3/lib/python3.9/site-packages/pandas/io/stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1727438286.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['pw_final', 'a_a_1','b_b_6a','b_b_7'].max().reset_index().rename(columns={'a_a_1':'household_size','b_b_6a':'num_hhd_in_dwelling','b_b_7':'own_dwelling'})
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1727

In [484]:
mtf_solar_df = pd.read_stata('./Datasets/Kenya_MTF_data/MTF_HH_Solar_Roster.dta')
with_weights_solar = pd.merge(mtf_solar_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['pw_final', 'c_position'].max().reset_index()
sum_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['c_131_pwr','c_133_batt'].sum().reset_index()
solar_hhs = pd.merge(max_vals_solar,sum_vals_solar,on = ['PARENT_KEY']).rename(columns={'c_133_batt':'solar_battery_capacity', 'c_131_pwr':'total_solar_rating','c_position':'num_solar'})

/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/2366092681.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['pw_final', 'c_position'].max().reset_index()
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/2366092681.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sum_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['c_131_pwr','c_133_batt'].sum().reset_index()


In [812]:
raw_mtf_data =pd.merge(total_survey,pd.merge(battery_hhs,solar_hhs, how='outer',on='PARENT_KEY').drop(columns=(['pw_final_x','pw_final_y'])), on='PARENT_KEY', how='left')

raw_mtf_data['weightedhousehold_inhabitants'] = raw_mtf_data.pw_final*raw_mtf_data.household_size
raw_mtf_data.loc[np.isnan(raw_mtf_data.num_hhd_in_dwelling),'num_hhd_in_dwelling'] = 0
raw_mtf_data['weighted_hh_per_dwelling'] = raw_mtf_data.pw_final*(raw_mtf_data.num_hhd_in_dwelling+1)
raw_mtf_data.loc[raw_mtf_data.own_dwelling==2,'own_dwelling'] = 0
raw_mtf_data['weighted_owned_dwelling'] = raw_mtf_data.pw_final*raw_mtf_data.own_dwelling
raw_mtf_data['weighted_rooms_occupied'] = raw_mtf_data.pw_final*raw_mtf_data.rooms_occupied
raw_mtf_data.loc[raw_mtf_data.grid_connection==2,'grid_connection'] = 0
raw_mtf_data['weighted_grid_connection'] = raw_mtf_data.pw_final*raw_mtf_data.grid_connection
raw_mtf_data.loc[raw_mtf_data.bank_account_held==2,'bank_account_held'] = 0
raw_mtf_data['weighted_bank_account'] = raw_mtf_data.pw_final*raw_mtf_data.bank_account_held
raw_mtf_data.loc[raw_mtf_data.mobile_money_acc_held==2,'mobile_money_acc_held'] = 0
raw_mtf_data['weighted_mobile_money'] = raw_mtf_data.pw_final*raw_mtf_data.mobile_money_acc_held
raw_mtf_data.loc[np.isnan(raw_mtf_data.num_batteries),'num_batteries'] = 0
raw_mtf_data['batteries_flag'] = 1
raw_mtf_data.loc[raw_mtf_data.num_batteries==0,'batteries_flag']=0
raw_mtf_data['weighted_batteries'] = raw_mtf_data.pw_final*raw_mtf_data.num_batteries
raw_mtf_data['weighted_battery_flag'] = raw_mtf_data.pw_final*raw_mtf_data.batteries_flag
raw_mtf_data.loc[np.isnan(raw_mtf_data.num_solar),'num_solar'] = 0
raw_mtf_data['solar_flag'] = 1
raw_mtf_data.loc[raw_mtf_data.num_solar==0,'solar_flag']=0
raw_mtf_data['weighted_solar'] = raw_mtf_data.pw_final*raw_mtf_data.num_solar
raw_mtf_data['weighted_solar_flag'] = raw_mtf_data.pw_final*raw_mtf_data.solar_flag
raw_mtf_data.loc[raw_mtf_data.generator_for_electricity==2,'generator_for_electricity'] = 0
raw_mtf_data['weighted_generator'] = raw_mtf_data.pw_final*raw_mtf_data.generator_for_electricity
raw_mtf_data['any_electricty'] = 1
raw_mtf_data.loc[(raw_mtf_data.solar_flag==0)&(raw_mtf_data.batteries_flag==0)&(raw_mtf_data.grid_connection==0)&(raw_mtf_data.generator_for_electricity==0),'any_electricty'] = 0
raw_mtf_data['weighted_electricity_use'] = raw_mtf_data.pw_final*raw_mtf_data.avg_electricity_consumption_kwh
raw_mtf_data['weighted_typical_hours_elect'] = raw_mtf_data.pw_final*raw_mtf_data.typical_hours_electricity
raw_mtf_data['weighted_worst_hours_elect'] = raw_mtf_data.pw_final*raw_mtf_data.worst_hours_electricity
raw_mtf_data['weighted_any_electricity'] = raw_mtf_data.pw_final*raw_mtf_data.any_electricty


In [644]:
#import geopandas
#district_gdf = geopandas.read_file("./Datasets/Shape files/ke_district_boundaries/ke_district_boundaries.shp")
#district_gdf.set_index('DISTNAME')
#level1_boundary = geopandas.read_file("./Datasets/Shape files/ken_adm_iebc_20191031_shp/ken_admbnda_adm1_iebc_20191031.shp")
#district_level1_lookup = pd.DataFrame(columns=['District','ADM1_EN'])
#for district in district_gdf.DISTNAME:
#    if district in (list(level1_boundary.ADM1_EN.str.upper())):
#        pair = [district, district]
#    else: 
#        overlap_value = level1_boundary.overlay(district_gdf[district_gdf.DISTNAME==district], how='intersection')
#        overlap_value['overlap_area']= overlap_value.area
#        pair = [district]+list(overlap_value.loc[overlap_value['overlap_area']==overlap_value['overlap_area'].max(),'ADM1_EN'].str.upper())#
#
#    district_level1_lookup.loc[len(district_level1_lookup)] = pair
#
#district_level1_lookup.to_csv('District_to_county_lookup.csv')

Need to tidy up the categories so they agree with the level 1 administrative regions

In [813]:

raw_mtf_data.loc[raw_mtf_data.dist.isin(['Bungoma South','Bungoma West','Bungoma North']),'dist'] = 'Bungoma'
raw_mtf_data.loc[raw_mtf_data.dist == 'Mandera Central','dist'] = 'Mandera'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nairobi West','Nairobi East','Nairobi North']),'dist'] = 'Nairobi'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nyandarua North','Nyandarua South']),'dist'] = 'Nyandarua'
raw_mtf_data.loc[raw_mtf_data.dist == 'Trans Nzoia West','dist'] = 'Trans Nzoia'
raw_mtf_data.loc[raw_mtf_data.dist == 'Taita','dist'] = 'Taita Taveta'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kakamega Central','Kakamega North','Mumias','Masaba']),'dist'] = 'Kakamega'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kisii Central','Kisii South']),'dist'] = 'Kisii'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nandi Central','Nandi East','Nandi North']),'dist'] = 'Nandi'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Meru Central','Imenti North','Igembe','Tigania']),'dist'] = 'Meru'

raw_mtf_data.loc[raw_mtf_data.dist.isin(['Muranga North','Muranga South']),'dist'] = "Murang'a"
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nyeri North','Nyeri South']),'dist'] = "Nyeri"
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Homabay']),'dist'] = 'Homa Bay'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Samburu Central','Samburu North']),'dist'] = 'Samburu'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Thika West']),'dist'] = 'Thika'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kisumu West']),'dist'] = 'Kisumu'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Turkana South','Turkana Central','Turkana North']),'dist'] = 'Turkana'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Gucha South']),'dist'] = 'Gucha'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kajiado North','Loitokitok']),'dist'] = 'Kajiado'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Narok North','Narok South']),'dist'] = 'Narok'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Laikipia East']),'dist'] = 'Laikipia'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Wajir South']),'dist'] = 'Wajir'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kilindini']),'dist'] = 'Mombasa'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Eldoret West','Eldoret East']),'dist'] = 'Uasin Gishu'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Mutomo','Wareng']),'dist'] = 'Uasin Gishu'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Pokot North']),'dist'] = 'Baringo'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nakuru North']),'dist'] = 'Nakuru'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nzaui']),'dist'] = 'Makueni'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Gatundu']),'dist'] = 'Kiambu'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kipkelion']),'dist'] = 'Kericho'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Chalbi']),'dist'] = 'Marsabit'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Pokot Central']),'dist'] = 'West Pokot'

In [814]:

raw_mtf_data.dist=raw_mtf_data.dist.str.upper()
level2_boundary = geopandas.read_file("./Datasets/Shape files/ken_adm_iebc_20191031_shp/ken_admbnda_adm2_iebc_20191031.shp")
level2_boundary_lookup = level2_boundary[['ADM1_EN','ADM2_EN']]
level2_boundary_lookup.ADM2_EN=level2_boundary_lookup.ADM2_EN.str.upper()
level2_boundary_lookup.ADM1_EN=level2_boundary_lookup.ADM1_EN.str.upper()
level1_info = level2_boundary.ADM1_EN.str.upper().drop_duplicates()
match_div2 = pd.merge(raw_mtf_data[['dist']].drop_duplicates(), level2_boundary_lookup, left_on='dist', right_on='ADM2_EN', how = 'left')
match_div2.loc[match_div2.dist.isin(list(level1_info)),'ADM1_EN']=match_div2.loc[match_div2.dist.isin(list(level1_info)),'dist']

district_county_lookup=pd.read_csv('District_to_county_lookup.csv').drop(columns='Unnamed: 0')


/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/960902721.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level2_boundary_lookup.ADM2_EN=level2_boundary_lookup.ADM2_EN.str.upper()
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/960902721.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level2_boundary_lookup.ADM1_EN=level2_boundary_lookup.ADM1_EN.str.upper()


In [815]:
with_district_lookup = pd.merge(match_div2,district_county_lookup,left_on='dist', right_on = 'District', how='left')
with_district_lookup.loc[pd.isna(with_district_lookup.ADM1_EN_x),'ADM1_EN_x'] = with_district_lookup.loc[pd.isna(with_district_lookup.ADM1_EN_x),'ADM1_EN_y']
#with_district_lookup.ADM1_EN_x= with_district_lookup.ADM1_EN_x.str.upper()

In [816]:
raw_mtf_data_new=pd.merge(raw_mtf_data,with_district_lookup.drop(columns=['ADM1_EN_y','ADM2_EN','District']).drop_duplicates(),on=['dist'],how='left').rename(columns={'ADM1_EN_x':'county'})
raw_mtf_data_new.loc[(pd.isna((raw_mtf_data_new.county)))&(raw_mtf_data_new.dist.isin(list(level1_info))),'county']=raw_mtf_data_new.loc[(pd.isna((raw_mtf_data_new.county)))&(raw_mtf_data_new.dist.isin(list(level1_info))),'dist']
raw_mtf_data_new.loc[(-raw_mtf_data_new.county.isin(list(level1_info)))&(raw_mtf_data_new.dist.isin(list(level1_info))),'county']=raw_mtf_data_new.loc[(-raw_mtf_data_new.county.isin(list(level1_info)))&(raw_mtf_data_new.dist.isin(list(level1_info))),'dist']

#raw_mtf_data_new.county = raw_mtf_data_new.county.str.upper()

In [817]:
grid_connected = raw_mtf_data_new[raw_mtf_data_new.grid_connection==1].copy()
not_grid_connected =raw_mtf_data_new[raw_mtf_data_new.grid_connection==0].copy()

not_grid_connected.loc[not_grid_connected.generator_for_electricity==2,'generator_for_electricity'] = 0



In [818]:
not_grid_means = not_grid_connected.groupby('county')[['distance_to_grid']].mean().reset_index().rename(columns={'distance_to_grid':'avg_distance_to_connect'})
not_grid_sums = not_grid_connected.groupby('county')[['weighted_generator','pw_final','weighted_batteries','weighted_battery_flag','weighted_solar_flag']].sum().reset_index().rename(columns={})
not_grid_sums['not_grid_generator_pct'] = not_grid_sums['weighted_generator']/not_grid_sums.pw_final
not_grid_sums['not_grid_battery_pct'] = not_grid_sums['weighted_battery_flag']/not_grid_sums.pw_final
not_grid_sums['not_grid_solar_pct'] = not_grid_sums['weighted_solar_flag']/not_grid_sums.pw_final
not_grid_data =pd.merge(not_grid_sums, not_grid_means, on='county',how='outer').drop(columns=['weighted_generator','pw_final','weighted_batteries','weighted_battery_flag','weighted_solar_flag'])

In [827]:
grid_sums = grid_connected.groupby('county')[['weighted_generator','pw_final','weighted_batteries','weighted_battery_flag','weighted_solar_flag','weighted_electricity_use','weighted_typical_hours_elect','weighted_worst_hours_elect']].sum().reset_index().rename(columns={})

grid_sums['grid_generator_pct'] = grid_sums['weighted_generator']/grid_sums.pw_final
grid_sums['grid_battery_pct'] = grid_sums['weighted_battery_flag']/grid_sums.pw_final
grid_sums['grid_solar_pct'] = grid_sums['weighted_solar_flag']/grid_sums.pw_final
grid_sums['avg_electricity_use_grid'] = grid_sums['weighted_electricity_use']/grid_sums.pw_final
grid_sums['avg_typical_hours_elect_grid'] = grid_sums['weighted_typical_hours_elect']/grid_sums.pw_final
grid_sums['avg_worst_hours_elect_grid'] = grid_sums['weighted_worst_hours_elect']/grid_sums.pw_final
grid_not_grid_data =pd.merge(grid_sums,not_grid_data, on='county',how='outer').drop(columns=['weighted_battery_flag','weighted_solar_flag','weighted_generator','pw_final','weighted_batteries','weighted_electricity_use','weighted_typical_hours_elect','weighted_worst_hours_elect'])

In [830]:
summed_dist_data=raw_mtf_data_new.groupby('county')[['pw_final', 'weightedhousehold_inhabitants','weighted_hh_per_dwelling','weighted_owned_dwelling','weighted_any_electricity','weighted_rooms_occupied','weighted_grid_connection','weighted_bank_account','weighted_mobile_money','weighted_generator','weighted_battery_flag','weighted_solar_flag']].sum().reset_index().rename(columns={'pw_final':'num_households','weightedhousehold_inhabitants':'total_inhabitants'})
summed_dist_data['avg_household_size'] = summed_dist_data.total_inhabitants/summed_dist_data.num_households
summed_dist_data['avg_household_per_dwelling'] = summed_dist_data.weighted_hh_per_dwelling/summed_dist_data.num_households
summed_dist_data['pct_own_dwelling'] = summed_dist_data.weighted_owned_dwelling/summed_dist_data.num_households
summed_dist_data['avg_rooms_occupied'] = summed_dist_data.weighted_rooms_occupied/summed_dist_data.num_households
summed_dist_data['percent_grid_connect'] = summed_dist_data.weighted_grid_connection/summed_dist_data.num_households
summed_dist_data['percent_any_elect'] = summed_dist_data.weighted_any_electricity/summed_dist_data.num_households
summed_dist_data['percent_bank_account'] = summed_dist_data.weighted_bank_account/summed_dist_data.num_households
summed_dist_data['percent_mobile_money'] = summed_dist_data.weighted_mobile_money/summed_dist_data.num_households
summed_dist_data['total_percent_batteries'] = summed_dist_data.weighted_battery_flag/summed_dist_data.num_households
summed_dist_data['total_percent_solar'] = summed_dist_data.weighted_solar_flag/summed_dist_data.num_households
summed_dist_data['total_percent_generator'] = summed_dist_data.weighted_generator/summed_dist_data.num_households

final_data=summed_dist_data.drop(columns=['weighted_grid_connection','weighted_battery_flag','weighted_generator','weighted_any_electricity','weighted_solar_flag','weighted_rooms_occupied','weighted_owned_dwelling','weighted_hh_per_dwelling','weighted_bank_account','weighted_mobile_money'])

mft_data_aggregated=pd.merge(final_data, grid_not_grid_data, on ='county',how='left')
mft_data_aggregated.to_csv("./Datasets/MFT_KENYA_SUMMARY.csv")